## Decorators
Decorators 以函数作为输入，修改函数的行为，然后返回一个新的函数

### python中是如何处理函数的

In [3]:
def func():
    print('hello from func')

In [4]:
func()

hello from func


In [5]:
new_func=func

In [6]:
new_func()

hello from func


In [7]:
print(new_func.__name__)

func


In [8]:
def call_func_twice(callback):
    callback()
    callback()
call_func_twice(func)

hello from func
hello from func


### 装饰器示例

In [13]:
def logging_decorator(func):
    def logger_wrapper(*args, **kargs):
        print(f"Before {func.__name__}")
        func(*args,**kargs)
        print(f"After {func.__name__}")
    return logger_wrapper

@logging_decorator
def sum(x,y):
    print(x+y)
sum(2,3)

Before sum
5
Before sum


### 有参数的装饰器

In [20]:
def logging_decorator(root):
    def _logging_decorator(func):
        def logger_wrapper(*args, **kargs):
            print(f"Before {func.__name__}: {root}")
            func(root,*args,**kargs)
            print(f"After {func.__name__}: {root}")
        return logger_wrapper
    return _logging_decorator

@logging_decorator(10)
def sum(root,x,y):
    print(root+x+y)
sum(2,3)

Before sum: 10
15
Before sum: 10


### 包装装饰器函数

In [5]:
import time
def logging_time(func):
    """Decorator that logs time"""
    def logger(*args, **kargs):
        """Function that logs time"""
        start=time.time()
        func(*args,**kargs)
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")
    return logger

@logging_time
def calculate_sum_n(n):
    return sum(range(n))

@logging_time
def say_hi(whom,greeting="Hello"):
    """Greet someone"""
    print(f"{greeting}, {whom}!")

calculate_sum_n(100000)
say_hi("john", greeting="hi")
# decoration 默认会修改被装饰函数的metadata，比如doc string，因为decoration说白了就是
# say_hello = logging_time(say_hello)
print(say_hi.__doc__)

Calling calculate_sum_n: 0.00367
hi, john!
Calling say_hi: 0.00005
Function that logs time


### 使用wraps保证被装饰函数元数据的正确性
为了避免被包装函数的元数据被污染，可以使用wraps

In [7]:
import time
from functools import wraps

def logging_time(func):
    """Decorator that logs time"""
    @wraps(func)
    def logger(*args, **kargs):
        """Function that logs time"""
        start=time.time()
        func(*args,**kargs)
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")
    return logger

@logging_time
def calculate_sum_n(n):
    return sum(range(n))

@logging_time
def say_hi(whom,greeting="Hello"):
    """Greet someone"""
    print(f"{greeting}, {whom}!")

calculate_sum_n(100000)
say_hi("john", greeting="hi")
print(say_hi.__doc__)

Calling calculate_sum_n: 0.00237
hi, john!
Calling say_hi: 0.00003
Greet someone


### 装饰器如何接受参数
为了使装饰器可以接受参数，需要再将被装饰函数包装一层

In [12]:
import time
from functools import wraps

def logging_time(unit):
    """Decorator that logs time"""
    def logger(func):
        @wraps(func)
        def inner_logger(*args, **kargs):
            """Function that logs time"""
            start=time.time()
            func(*args,**kargs)
            scaling = 1000 if unit == "ms" else 1
            print(f"Calling {func.__name__}: {(time.time() - start) * scaling:.5f} {unit}")
        return inner_logger
    return logger

@logging_time("ms")
def calculate_sum_n(n):
    return sum(range(n))

@logging_time("s")
def say_hi(whom,greeting="Hello"):
    """Greet someone"""
    print(f"{greeting}, {whom}!")

calculate_sum_n(100000)
say_hi("john", greeting="hi")

Calling calculate_sum_n: 3.04294 ms
hi, john!
Calling say_hi: 0.00004 s


### 多个装饰器，顺序很重要

In [4]:
import time
from functools import wraps

def repeat(func):
    """Decorator that repeats function call twice"""
    def repeater(*args,**kargs):
        func(*args,**kargs)
        func(*args,**kargs)
    return repeater

def logging_time(unit):
    """Decorator that logs time"""
    def logger(func):
        @wraps(func)
        def inner_logger(*args, **kargs):
            """Function that logs time"""
            start=time.time()
            func(*args,**kargs)
            scaling = 1000 if unit == "ms" else 1
            print(f"Calling {func.__name__}: {(time.time() - start) * scaling:.5f} {unit}")
        return inner_logger
    return logger



@logging_time("s")
@repeat
def say_hi(whom,greeting="Hello"):
    """Greet someone"""
    print(f"{greeting}, {whom}!")

@repeat
@logging_time("s")
def say_hello(whom,greeting="Hello"):
    """Greet someone"""
    print(f"{greeting}, {whom}!")
say_hi("John")
say_hello("Jone")

Hello, John!
Hello, John!
Calling repeater: 0.00023 s
Hello, Jone!
Calling say_hello: 0.00003 s
Hello, Jone!
Calling say_hello: 0.00002 s


### Class Based Decorators

In [11]:
class Logger:
    def __init__(self, n):
        self.n=n
    def __call__(self, func):
        def inner_logger(*args, **kargs):
            print(f"Before {func.__name__}")
            func(self.n,*args,**kargs)
            print(f"After {func.__name__}")
        return inner_logger

@Logger(2)
def sum(*args):
    print(*args)
sum(2,3)  

Before sum
2 2 3
After sum
